# mein bericht

das ist meine abgabe

In [1]:
import requests
from eliza import eliza
import os
import json
import sox
from wendel_util import file_update
import dialog_util as util
import emorec

In [2]:
file_update()
vaccinations = open('vaccinations.json')
vaccinations = json.load(vaccinations)

Up To Date


In [3]:
util.init_google()

In [4]:
def normalize(in_s):
    # e.g. remove stopwords, lemmatization, stemming, 
    return in_s.lower()

In [5]:
def do_input():
    return util.speech_input()

In [6]:
def semantic(input_s):
    semantics = {'state':'', 'vaccine':'', 'answer':0}
    for key in util.states_d.keys():
        if key in input_s:
            semantics['state'] = util.states_d[key]
            break
    for key in util.vaccines_d.keys():
        if key in input_s:
            semantics['vaccine'] =  util.vaccines_d[key]
            break
    return semantics

In [7]:
# expects semantics: semantics[0] == bundesland, semantics[1] == impfstoff 
def data(semantics):
    s = semantics['state']
    v = semantics['vaccine']
    if s: # state given
        if s != 'DE':
            if v: # and vaccine given
                semantics['answer'] = vaccinations["data"]["states"][s]['vaccination'][v]
            else: # all vaccines for state
                semantics['answer'] = vaccinations["data"]["states"][s]['vaccinated']
        else:
            if v: # and vaccine given
                semantics['answer'] = vaccinations["data"]['vaccination'][v]
            else: # all vaccines for Germany
                semantics['answer'] = vaccinations['data']['vaccinated']
    else: # no state
        if v: # but vaccine
            semantics['answer'] = vaccinations["data"]['vaccination'][v]
        else: # nothing given
            semantics['answer'] = None
    return semantics

In [8]:
def output(semantics, inputs):
    ret = ''
    s = semantics['state']
    v = semantics['vaccine']
    a = semantics['answer']
    if s: # state given
        s = util.state_names[s]
        if v: # and vaccine given
            v = util.vaccine_names[v]
            ret = 'Die Impfungen für {} mit {} sind {}'.format(s, v, a)
        else: # all vaccines for state
            ret = 'Die Impfungen für {} sind {}'.format(s, a)
    else: # no state
        if v: # but vaccine
            v = util.vaccine_names[v]
            ret = 'Die Impfungen in Deutschland mit {} sind {}'.format(v, a)
        else: # nothing given
            ret =  elz.respond(inputs)
    return ret

In [9]:
def init_eliza():
    root = r'/Users/shushanamanakhimova/S_Dialog/'
    elz = eliza.Eliza()
    elz.load(root+'/eliza/deutsch.txt')
    return elz

In [10]:
def output_s(text):
    print('output: '+text)
    util.tts(text)

In [11]:
emo_dict = {'happiness':'freundlich', 'neutral': 'wie immer', 'anger': 'ärgerlich', 'sadness': 'traurig', 
            'fear': 'ängstlich', 'boredom':'gelangweilt', 'disgust':'angeekelt'}

In [12]:
def dialogmanager():
    output_s("Frag was!")
    input_s = do_input()
    input_s = normalize(input_s)
    while input_s and input_s != 'fertig':
        emotion = emoRec.classify(filename)[0]
        emotion_g = emo_dict[emotion]
        output_s('ich merke du bist '+emotion_g)
        semantics = semantic(input_s)
        semantics = data(semantics)
        out_string = output(semantics, input_s)
        output_s(out_string)
        input_s = do_input()
        input_s = normalize(input_s)

In [13]:
filename = 'myfile.wav'
elz = init_eliza()
emoRec = emorec.EmoRec()
dialogmanager()

output: Frag was!
[[ 0.0000000e+00]
 [ 0.0000000e+00]
 [ 0.0000000e+00]
 ...
 [-3.0332692e-05]
 [ 3.3720869e-05]
 [ 9.1965077e-05]]
Transcript 0: wie geht's
output: ich merke du bist traurig
output: Ich bin nicht sicher, ob ich dich verstanden habe.
[[0.        ]
 [0.        ]
 [0.        ]
 ...
 [0.11251903]
 [0.10367163]
 [0.09281435]]
Transcript 0: diese
output: ich merke du bist traurig
output: Erzähl mir weiter.
[[ 0.0000000e+00]
 [ 0.0000000e+00]
 [ 0.0000000e+00]
 ...
 [-1.0997682e-05]
 [ 9.4158619e-05]
 [ 2.8024639e-05]]
Transcript 0: verstehst du mich
output: ich merke du bist angeekelt
output: Wir reden über dich, nicht über mich.
[[0.        ]
 [0.        ]
 [0.        ]
 ...
 [0.00409326]
 [0.00415841]
 [0.00427732]]
Transcript 0: ich möchte über dich sprechen
output: ich merke du bist freundlich
output: Was würde es dir bedeuten, wenn du diesen Wunsch realisieren würdest ?
[[0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 ...
 [5.75581689e-05]
 [6.97043943e-05]
 [1.17

AttributeError: 'NoneType' object has no attribute 'lower'